函数大概解释：一个保存了固定字典和大小的简单查找表。这个模块常用来保存词嵌入和用下标检索它们。模块的输入是一个下标的列表，输出是对应的词嵌入。相当于随机生成了一个tensor，可以把它看作一个查询表，其size为[num_embeddings，embedding_dim] 。其中num_embeddings是查询表的大小，embedding_dim是每个查询向量的维度，这里看起来很抽象，可以看下面的示例来进行理解。

这是一个矩阵类，里面初始化了一个随机矩阵，矩阵的长是字典的大小，宽是用来表示字典中每个元素的属性向量，向量的维度根据你想要表示的元素的复杂度而定。类实例化之后可以根据字典中元素的下标来查找元素对应的向量。输入下标0，输出就是embeds矩阵中第0行。

```python
nn.Embedding(num_embeddings: int, 
             embedding_dim: int, 
             padding_idx: Optional[int] = None,
             max_norm: Optional[float] = None, 
             norm_type: float = 2., 
             scale_grad_by_freq: bool = False,
             sparse: bool = False, 
             _weight: Optional[Tensor] = None,
             device=None, dtype=None) -> None:
```


### 一个简单的查找表，用于存储固定字典和大小的嵌入。

- 该模块通常用于存储词嵌入并使用索引检索它们。
     模块的输入是索引列表，输出是对应的
     词嵌入。
        
    - 参数：
        - num_embeddings (int): 嵌入字典的大小
        - embedding_dim (int)：每个嵌入向量的大小
        - padding_idx (int, optional): 如果指定，:attr:`padding_idx` 处的条目不会对梯度产生影响；
        因此，:attr:`padding_idx` 处的嵌入向量在训练期间不会更新，
        即它仍然是一个固定的“垫”。对于一个新建的 Embedding，
        padding_idx 处的嵌入向量将默认为全零，
        但可以更新为另一个值以用作填充向量。
        - max_norm (float, optional): 如果给定，每个嵌入向量的范数大于:attr:`max_norm`
        被重新规范化为具有范数:attr:`max_norm`。
        - norm_type (float, optional): 为 :attr:`max_norm` 选项计算的 p-norm 的 p。默认“2”。
        - scale_grad_by_freq (boolean, optional): 如果给定，这将通过频率的倒数来缩放梯度
        小批量中的单词。默认“假”。
        - sparse (bool, optional): If ``True``, 梯度 w.r.t. :attr:`weight` 矩阵将是一个稀疏张量。
        有关稀疏渐变的更多详细信息，请参阅注释。
    
    - 属性：
         - 权重（Tensor）：形状模块的可学习权重（num_embeddings，embedding_dim）
         从 :math:$\mathcal{N}(0, 1)$ 初始化


    - 形状：
         - 输入：:math:`(*)`、IntTensor 或 LongTensor 任意形状，包含要提取的索引
         - 输出：:math:`(*, H)`，其中 `*` 是输入形状，:math:$H=\text{embedding\_dim}$

    - 注意：
         - 请记住，只有有限数量的优化器支持
         稀疏梯度：目前是 :class:`optim.SGD`（`CUDA` 和 `CPU`），
         :class:`optim.SparseAdam` (`CUDA` 和 `CPU`) 和 :class:`optim.Adagrad` (`CPU`)

    -  注意:
          - 当 :attr:`max_norm` 不是 `None` 时，:class:`Embedding` 的 forward 方法会修改
                   :attr:`weight` 原位张量。 由于梯度计算所需的张量不能
                   就地修改，在之前对“Embedding.weight”执行可微操作
                   调用 Embedding 的 forward 方法需要克隆 Embedding.weight 时
                   :attr:`max_norm` 不是“无”。


In [71]:
import torch
from torch import nn

实例一：创建查询矩阵并使用它做Embedding：

In [72]:
embedding = nn.Embedding(5, 3)  # 定义一个具有5个单词，维度为3的查询矩阵
print(embedding.weight)  # 展示该矩阵的具体内容
test = torch.LongTensor([[0, 2, 0, 1],
                         [1, 3, 4, 4]])  # 该test矩阵用于被embed，其size为[2, 4]
# 其中的第一行为[0, 2, 0, 1]，表示获取查询矩阵中ID为0, 2, 0, 1的查询向量
# 可以在之后的test输出中与embed的输出进行比较
test = embedding(test)
print(test.size())  # 输出embed后test的size，为[2, 4, 3]，增加的3，是因为查询向量的维度为3
print(test)  # 输出embed后的test的内容

Parameter containing:
tensor([[-0.2102,  0.6403,  1.9413],
        [-0.1649,  0.6185, -0.7571],
        [-0.3988, -0.2256, -0.9698],
        [-0.5868, -1.6040,  0.3543],
        [-1.4321,  0.0282, -1.4700]], requires_grad=True)
torch.Size([2, 4, 3])
tensor([[[-0.2102,  0.6403,  1.9413],
         [-0.3988, -0.2256, -0.9698],
         [-0.2102,  0.6403,  1.9413],
         [-0.1649,  0.6185, -0.7571]],

        [[-0.1649,  0.6185, -0.7571],
         [-0.5868, -1.6040,  0.3543],
         [-1.4321,  0.0282, -1.4700],
         [-1.4321,  0.0282, -1.4700]]], grad_fn=<EmbeddingBackward0>)


可以看出创建了一个具有5个ID(可以理解为拥有5个词的词典)的查询矩阵，每个查询向量的维度是3维，然后用一个自己需要Embedding的矩阵与之计算，其中的内容就是需要匹配的ID号，注意！如果需要Embedding的矩阵中的查询向量不为1，2这种整数，而是1.1这种浮点数，就不能与查询向量成功匹配，会报错，且如果矩阵中的值大于了查询矩阵的范围，比如这里是5，也会报错。

实例二：寻找查询矩阵中特定ID(词)的查询向量(词向量)：

In [73]:
# 访问某个ID，即第N个词的查询向量(词向量)
embedding(torch.LongTensor([3]))  # 这里表示查询第3个词的词向量

tensor([[-0.5868, -1.6040,  0.3543]], grad_fn=<EmbeddingBackward0>)

*实例三：输出的hello这个词的word embedding

In [74]:
from torch.autograd import Variable
word_to_ix = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2, 5)
hello_idx = torch.LongTensor([word_to_ix['hello']])
hello_idx = Variable(hello_idx)
hello_embed = embeds(hello_idx)
hello_embed

tensor([[ 1.3442, -0.7834,  2.0002, -2.0970, -0.5910]],
       grad_fn=<EmbeddingBackward0>)

官方示例

In [75]:
# num_embeddings, embedding_dim, max_norm
n, d, m = 3, 5, 7
embedding = nn.Embedding(n, d, max_norm=True) 
W = torch.randn((m, d), requires_grad=True)
idx = torch.tensor([1, 2])
a = embedding.weight.clone() @ W.t()  # weight must be cloned for this to be differentiable
b = embedding(idx) @ W.t()  # modifies weight in-place
out = (a.unsqueeze(0) + b.unsqueeze(1))
loss = out.sigmoid().prod()
loss.backward()

In [76]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
embedding(input)

tensor([[[-1.3583, -0.7382,  0.2429],
         [ 0.3089,  1.8333,  2.1872],
         [-1.6570,  0.6716, -1.1453],
         [-0.2945,  1.8201, -0.4705]],

        [[-1.6570,  0.6716, -1.1453],
         [ 0.3928, -0.9647,  1.0261],
         [ 0.3089,  1.8333,  2.1872],
         [ 0.1122, -0.6475,  1.0859]]], grad_fn=<EmbeddingBackward0>)

In [77]:
# example with padding_idx
embedding = nn.Embedding(10, 3, padding_idx=0)
input = torch.LongTensor([[0,2,0,5]])
embedding(input)

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.6057, -1.7844, -0.8746],
         [ 0.0000,  0.0000,  0.0000],
         [-1.1813, -0.2832,  1.9689]]], grad_fn=<EmbeddingBackward0>)

In [78]:
# example of changing `pad` vector
padding_idx = 0
embedding.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [-1.4273, -0.7405,  0.1696],
        [ 0.6057, -1.7844, -0.8746],
        [-1.4139, -1.1599,  2.3117],
        [-0.0528, -0.1413,  1.0461],
        [-1.1813, -0.2832,  1.9689],
        [ 0.1231, -0.4302,  3.2592],
        [ 0.8762,  1.2861, -1.5752],
        [ 0.9645,  0.7281, -1.8929],
        [ 1.6235, -0.8477,  0.6434]], requires_grad=True)

In [79]:
with torch.no_grad():
    embedding.weight[padding_idx] = torch.ones(3)
embedding.weight

Parameter containing:
tensor([[ 1.0000,  1.0000,  1.0000],
        [-1.4273, -0.7405,  0.1696],
        [ 0.6057, -1.7844, -0.8746],
        [-1.4139, -1.1599,  2.3117],
        [-0.0528, -0.1413,  1.0461],
        [-1.1813, -0.2832,  1.9689],
        [ 0.1231, -0.4302,  3.2592],
        [ 0.8762,  1.2861, -1.5752],
        [ 0.9645,  0.7281, -1.8929],
        [ 1.6235, -0.8477,  0.6434]], requires_grad=True)